## Order of colab reference notebooks:
1. Reddit Data Scrapper.ipynb \
2. Pipeline_for_BOW.ipynb  \
3. dataset_classification.ipynb \
4. Toxic_Span_Detection_BiLSTM+CRF_model.ipynb \

# Dataset Collection

Installing necessary libraries

In [ ]:
# !pip install praw
# !pip install asyncpraw
# import praw
import tqdm
import asyncpraw
import re
# from praw.models import MoreComments
import pandas as pd
import asyncio

Fetching news subreddit data

In [ ]:
reddit = asyncpraw.Reddit(client_id='q6HKrPg90SJKqevoa6i8Tw', client_secret='wlS_eetUTdF7oddrOLdABpBkQW14EQ', user_agent='nlp-toxic')
news_subreddit = await reddit.subreddit('news')

In [ ]:
posts = []
comments_reddit = []

async for post in news_subreddit.hot(limit=200):
    posts.append([post.id])
posts = pd.DataFrame(posts, columns=["id"])


Fetching the comments from the news subreddit and preprocessing 

In [ ]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)


comments_reddit=[]
for subid in posts["id"][0:1]:
    submission = await reddit.submission(subid)
    print(subid)
    await submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        sentencing = comment.body
        #Removing spaces and new lines in a single comment
        sentencing = sentencing.replace('\n','')
        sentencing = sentencing.replace('\t','')
        sentencing = sentencing.strip()
        #Removing any urls
        # sentencing = sentencing.replace('https\S+|www.\S+', '')
        sentencing = re.sub(r'https\S+|www.\S+','',sentencing)
        #1 or more white spaces with single whitespace
        sentencing = re.sub(r'\s+', ' ', sentencing)
        # sentencing = emoji_pattern.sub(r'',sentencing)
        comments_reddit.append(sentencing)

Save the comments to a csv file

In [ ]:
comments_subreddit = pd.DataFrame(comments_reddit, columns=["comments"])
comments_subreddit.to_csv("news_comments.csv")